In [ ]:
import mlflow
from azureml.core import Workspace

from sklearn.linear_model import Ridge
import mlflow.sklearn

In [ ]:
def get_workspace():
    """
    MLLab Área de trabajo del servicio Machine Learning
    a21d8af7-720a-451d-9010-9defcff47587 Suscription id
    mllab7849892444 Microsoft.Insights/components
    mllab8506144354 Microsoft.KeyVault/vaults
    mllab6951918960 Microsoft.Storage/storageAccounts
    MLLab Microsoft.MachineLearningServices/workspace
    MLLab-resource-group Resource group
    MLLab-K8s Nombre cluster kubernetes
              (MLLab-K8s es id de recurso, nombre del proceso myaks)
    """
    ws = Workspace.get(name="MLLab", subscription_id='a21d8af7-720a-451d-9010-9defcff47587', resource_group='MLLab-resource-group')
    return ws

In [ ]:
ws = get_workspace()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [ ]:
def get_training_data():        
  sample_files_path = os.environ.get('SAMPLE_FILES_PATH','.')
  trainingDataFilePath = sample_files_path + '/articulos_ml.csv'
  data = pd.read_csv(trainingDataFilePath)
  # veamos cuantas dimensiones y registros contiene (son 161 registros con 8 columnas)
  # print(data.shape)
  # Vamos a RECORTAR los datos en la zona donde se concentran más los puntos
  filtered_data = data[(data['Word count'] <= 3500) & (data['# Shares'] <= 80000)]
  return filtered_data

In [ ]:

def train(alpha_arg, training_data):
  dataX = training_data[["Word count"]]
  X_train = np.array(dataX)
  y_train = training_data['# Shares'].values
  # Creamos el objeto de Regresión Ridge
  regr = Ridge(alpha=alpha_arg)
  # Entrenamos nuestro modelo (worst model ever)
  regr.fit(X_train, y_train)
  return regr  

experiment_name = 'experiment_mite_1'
mlflow.set_experiment(experiment_name)

with mlflow.start_run():
  alpha = 0.01 
  regression_model = train(alpha)    
  mlflow.log_metric('alpha', alpha)
  # Save the model to the outputs directory for capture
  model_save_path = "mite_models"
  mlflow.sklearn.log_model(regression_model, model_save_path)
